In [ ]:
!pip install yt-dlp moviepy whisper transformers
!apt-get install ffmpeg
!pip install openai-whisper

In [ ]:
import whisper
from transformers import pipeline
from moviepy.editor import VideoFileClip
import yt_dlp
import os
import torch

# Função para baixar o vídeo usando yt-dlp
def download_video(url):
    ydl_opts = {
        'format': 'bestvideo+bestaudio',
        'outtmpl': 'video.%(ext)s',
        'merge_output_format': 'mp4',  # Força a saída em formato mp4
        'postprocessors': [{
            'key': 'FFmpegVideoConvertor',
            'preferedformat': 'mp4',  # Converte o vídeo para mp4 após o download
        }],
        'keepvideo': True,  # Mantém os arquivos originais
    }
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([url])
    if os.path.exists('video.mp4'):
        print("Vídeo baixado com sucesso.")
        return 'video.mp4'
    else:
        raise FileNotFoundError("Erro: o arquivo 'video.mp4' não foi encontrado após o download.")

# Função para extrair o áudio do vídeo
def extract_audio(video_path, audio_output):
    if os.path.exists(video_path):
        video = VideoFileClip(video_path)
        video.audio.write_audiofile(audio_output)
        print("Áudio extraído com sucesso.")
    else:
        raise FileNotFoundError(f"Erro: o arquivo '{video_path}' não foi encontrado.")

# Função para transcrever o áudio
def transcribe_audio(audio_path):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model = whisper.load_model("base").to(device)
    result = model.transcribe(audio_path)
    print("Transcrição concluída.")
    return result['text']

# Função para dividir o texto em partes menores
def split_text(text, max_length):
    words = text.split()
    current_length = 0
    current_chunk = []

    for word in words:
        current_length += len(word) + 1  # +1 para o espaço
        if current_length > max_length:
            yield ' '.join(current_chunk)
            current_chunk = [word]
            current_length = len(word) + 1
        else:
            current_chunk.append(word)

    if current_chunk:
        yield ' '.join(current_chunk)

# Função para resumir o texto
def summarize_text(text):
    summarizer = pipeline("summarization")
    summaries = []
    for chunk in split_text(text, 512):  # Certificar que cada chunk tem no máximo 512 tokens
        summary = summarizer(chunk, max_length=130, min_length=30, do_sample=False)
        summaries.append(summary[0]['summary_text'])
    print("Resumos concluídos.")
    return ' '.join(summaries)

# Entrada do usuário: URL do vídeo
video_url = input("Digite o URL do vídeo: ")

# 1. Baixar o vídeo
video_path = download_video(video_url)
audio_path = "audio.wav"

# 2. Extrair o áudio
extract_audio(video_path, audio_path)

# 3. Transcrever o áudio
transcription = transcribe_audio(audio_path)
print("Transcrição: ", transcription)

# 4. Resumir a transcrição
summary = summarize_text(transcription)
print("Resumo: ", summary)

# Remover arquivos temporários (opcional)
os.remove(video_path)
os.remove(audio_path)
